In [ ]:
import gc  
import os  
import time  
import warnings 
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import lightgbm as lgb  
import xgboost as xgb
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit
import polars as pl
import psutil
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_offline = False 
LGB = False
XGB = True
NN = False
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  


# **Utilities**

In [ ]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

def get_used_memory_MB():
    return np.round(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2,2)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
    return df


# Data Loading






In [ ]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
# df_shape = df.shape
print(df.shape)
print('Using total Memory=', get_used_memory_MB(),"MB")

**Outlier removal**

In [ ]:
# outlier_days = [33,
#  48,
#  58,
#  83,
#  86,
#  96,
#  119,
#  122,
#  158,
#  193,
#  196,
#  208,
#  221,
#  225,
#  283,
#  328,
#  334,
#  346,
#  404,
#  407,
#  438,
#  453,
#  458,
#  470,
#  474]

# outlier_days = [33,
#  35,
#  83,
#  86,
#  96,
#  119,
#  122,
#  158,
#  193,
#  196,
#  208,
#  221,
#  225,
#  283,
#  322,
#  328,
#  334,
#  346,
#  404,
#  407,
#  458,
#  470,
#  474]

# df = df.query('date_id not in @outlier_days').reset_index(drop=True)

 # Parallel Triplet Imbalance Calculation

In [ ]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features


# Feature Generation Functions 

In [ ]:
def dfrank(newdf): # 添加基础排名因子
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    columns=[column for column in newdf.columns if ((column in prices)or(column in sizes))]
    for column in columns:
        # 从小到大排名【测试下双排名有效果是因为加上了na_option='bottom'的处理机制还是因为实现的双排名方案】
        newdf=pd.concat([newdf,(newdf[str(column)].rank(method="max", ascending=False,na_option='bottom')/len(newdf)).rename(f"{str(column)}_rank")], axis=1) # 从大到小排序
        # 从大到小排名
        newdf=pd.concat([newdf,(newdf[str(column)].rank(method="max", ascending=True,na_option='bottom')/len(newdf)).rename(f"{str(column)}_rerank")], axis=1) # 从大到小排序
    return newdf

def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
   
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
    
    # Calculate various statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        

    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1,3,5,10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'weighted_wap','price_spread']:
        for window in [1,3,5,10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    
    #V4 feature
    for window in [3,5,10]:
        df[f'price_change_diff_{window}'] = df[f'bid_price_diff_{window}'] - df[f'ask_price_diff_{window}']
        df[f'size_change_diff_{window}'] = df[f'bid_size_diff_{window}'] - df[f'ask_size_diff_{window}']

    # New Features
    df = df.groupby(['date_id','seconds_in_bucket']).apply(dfrank) # 计算排名因子【之前得分最好的方案没有这个因子】
    #匹配失败数量和匹配成功数量的比率
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    #供需市场的差额
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    #供需市场总和
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    #供需价格的均值
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    #整体数据情况
    median_sizes = df.groupby('stock_id')['bid_size'].median() + df.groupby('stock_id')['ask_size'].median()
    std_sizes = df.groupby('stock_id')['bid_size'].std() + df.groupby('stock_id')['ask_size'].std()
    max_sizes = df.groupby('stock_id')['bid_size'].max() + df.groupby('stock_id')['ask_size'].max()
    min_sizes = df.groupby('stock_id')['bid_size'].min() + df.groupby('stock_id')['ask_size'].min()
    mean_sizes = df.groupby('stock_id')['bid_size'].mean() + df.groupby('stock_id')['ask_size'].mean()
    first_sizes = df.groupby('stock_id')['bid_size'].first() + df.groupby('stock_id')['ask_size'].first()
    last_sizes = df.groupby('stock_id')['bid_size'].last() + df.groupby('stock_id')['ask_size'].last()
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())    
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())
    del median_sizes,std_sizes,max_sizes,min_sizes,mean_sizes,first_sizes,last_sizes
    gc.collect()
    #feats
    df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
        
    #V5 - rolling diff
    # Convert from pandas to Polars
    pl_df = pl.from_pandas(df)

    #Define the windows and columns for which you want to calculate the rolling statistics
    windows = [3, 5, 10]
    columns = ['ask_price', 'bid_price', 'ask_size', 'bid_size']

    # prepare the operations for each column and window
    group = ["stock_id"]
    expressions = []

    # Loop over each window and column to create the rolling mean and std expressions
    for window in windows:
        for col in columns:
            rolling_mean_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_mean(window)
                .over(group)
                .alias(f'rolling_diff_{col}_{window}')
            )

            rolling_std_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_std(window)
                .over(group)
                .alias(f'rolling_std_diff_{col}_{window}')
            )

            expressions.append(rolling_mean_expr)
            expressions.append(rolling_std_expr)

    # Run the operations using Polars' lazy API
    lazy_df = pl_df.lazy().with_columns(expressions)

    # Execute the lazy expressions and overwrite the pl_df variable
    pl_df = lazy_df.collect()

    # Convert back to pandas if necessary
    df = pl_df.to_pandas()
    del pl_df,lazy_df
    gc.collect()
    
    
    df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    
    for col in df.columns:
        df[col] = df[col].replace([np.inf, -np.inf], 0)
        
    gc.collect()
    return df

def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  
    df["minute"] = df["seconds_in_bucket"] // 60  
    df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    
    # new feature
    df["dom"] = df["date_id"] % 21 # Day of the month
    df["doq"] = df["date_id"] % 63 # Day of the quarter
    df["doy"] = df["date_id"] % 252 # Day of the year
    df['before_5min'] = 1
    df.loc[df['seconds_in_bucket']>=300,'before_5min'] = -1
    df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    # Cyclical encoding for day of the week
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 5)  # 5 is the number of weekdays
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 5)
    # Cyclical encoding for minute of the day
    df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)  # 1440 minutes in a day
    df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)
    df['second_sin'] = np.sin(2 * np.pi * df['seconds'] / 60)  # 60 seconds in a minute
    df['second_cos'] = np.cos(2 * np.pi * df['seconds'] / 60)
    
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

def generate_all_features(df):
    # Select relevant columns for feature generation
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    gc.collect()
    
    # Generate imbalance features
    df = imbalance_features(df)
    gc.collect() 
    df = other_features(df)
    gc.collect()  
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    return df[feature_name]


In [ ]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
weights = {int(k):v for k,v in enumerate(weights)}

## Data Splitting

In [ ]:
if is_offline:
    
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
    
else:
    df_train = df
    print("Online mode")
print('Using total Memory=', get_used_memory_MB(),"MB")

In [ ]:
if is_train:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }
    if is_offline:
        df_train_feats = generate_all_features(df_train)
        print("Build Train Feats Finished.")
        df_valid_feats = generate_all_features(df_valid)
        print("Build Valid Feats Finished.")
        df_valid_feats = reduce_mem_usage(df_valid_feats)
    else:
        df_train_feats = generate_all_features(df_train)
        print("Build Online Train Feats Finished.")

    df_train_feats = reduce_mem_usage(df_train_feats)

# Print number of features
feature_columns = list(df_train_feats.columns)
print(f"Features = {len(feature_columns)}")
print('Using total Memory=', get_used_memory_MB(),"MB")

In [ ]:
# corr = df_train_feats.corr()
# # Drop highly correlated features (37->30)
# columns = np.full((corr.shape[0],), True, dtype=bool)
# for i in range(corr.shape[0]):
#     for j in range(i+1, corr.shape[0]):
#         if corr.iloc[i,j] >= 0.99:
#             if columns[j]:
#                 columns[j] = False

# feature_columns = df_train_feats.columns[columns].values
# drop_columns = df_train_feats.columns[columns == False].values
# print(feature_columns)
# print('-'*73)
# print(drop_columns)

# gc.collect()

# **Model Training**

# **LGB**

In [ ]:

if LGB:
    import numpy as np
    import lightgbm as lgb
    
    lgb_params = {
        "objective": "mae",
        "n_estimators": 6000,
        "num_leaves": 256,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
#         "learning_rate": 0.00871,
        "learning_rate": 0.01,
        'max_depth': 11,
        "device": "gpu",
        "n_jobs": 4,
        "verbosity": -1,
        "importance_type": "gain",
#         "reg_alpha": 0.1,
        "reg_alpha": 0.2,
        "reg_lambda": 3.25
    }

    feature_columns = list(df_train_feats.columns)
    print(f"Features = {len(feature_columns)}")
    #print(f"Feature length = {len(feature_columns)}")

    num_folds = 5
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues' 
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train_feats[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train_feats[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

        print(f"Fold {i+1} Model Training")

        # Train a LightGBM model for the current fold
        lgb_model = lgb.LGBMRegressor(**lgb_params)
        lgb_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )
        
#         cbt_model = cbt.CatBoostRegressor(objective='MAE', iterations=5000,bagging_temperature=0.5,
#                                 colsample_bylevel = 0.7,learning_rate = 0.065,
#                                 od_wait = 25,max_depth = 7,l2_leaf_reg = 1.5,
#                                 min_data_in_leaf = 1000,random_strength=0.65,
#                                 verbose=0,use_best_model=True,task_type='CPU')
#         cbt_model.fit(
#             df_fold_train[feature_columns],
#             df_fold_train_target,
#             eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)]
#         )
        
#         models_cbt.append(cbt_model)

        models.append(lgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        fold_predictions = lgb_model.predict(df_fold_valid[feature_columns])
        fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
        scores.append(fold_score)
        print(f":LGB Fold {i+1} MAE: {fold_score}")
        #------------CBT--------------#
#         fold_predictions = cbt_model.predict(df_fold_valid[feature_columns])
#         fold_score_cbt = mean_absolute_error(fold_predictions, df_fold_valid_target)
#         scores.append(fold_score_cbt)
#         print(f"CBT Fold {i+1} MAE: {fold_score_cbt}")

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

    # Update the lgb_params with the average best iteration
    final_model_params = lgb_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1

    for i in range(num_model):
        final_model = lgb.LGBMRegressor(**final_model_params)
        final_model.fit(
            df_train_feats[feature_columns],
            df_train['target'],
            callbacks=[
                lgb.callback.log_evaluation(period=100),
            ],
        )
        # Append the final model to the list of models
        models.append(final_model)
        
    # Calculate and print the average MAE across all folds
    average_mae = np.mean(scores)
    print(f"Average MAE across all folds: {average_mae}")

# XGB

In [ ]:
if XGB:
    
    xgb_params = {
        "objective": "reg:squarederror",
        "n_estimators": 6000,
        #"n_estimators": 30,
        "random_state": 42,
        "subsample": 0.6,
        "colsample_bytree": 0.5,
        #"learning_rate": 0.00971,
        "learning_rate": 0.01,
        'max_depth': 11,
#         "n_jobs": -1,
#         "device": "cpu",
        "verbosity": 3,
        #"reg_alpha": 0.1,
        "reg_alpha": 0.5,
        "reg_lambda": 1,
        "tree_method":'gpu_hist',
        "predictor":'gpu_predictor',
    }


#     logger.info(f"xgb_params: {xgb_params}")

    feature_columns = list(df_train_feats.columns)
#     print(f"Features = {len(feature_columns)}")
    print(f"Feature length = {len(feature_columns)}")

    num_folds = 10
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues'     
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values
    
    time_cost_list = []

    for i in range(num_folds):
        break
        now_time = time.time()
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train_feats[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train_feats[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

#         logger.info(f"Fold {i+1} Model Training")
        print(f"Fold {i+1} Model Training")

        # Train a LightGBM model for the current fold
        xgb_model = xgb.XGBRegressor(**xgb_params)
        xgb_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            eval_metric='mae',
            early_stopping_rounds=100,
            verbose=100,
            # verbose=True,
        )
    
#         lgb_model = lgb.LGBMRegressor(**lgb_params)
#         lgb_model.fit(
#             df_fold_train[feature_columns],
#             df_fold_train_target,
#             eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
#             callbacks=[
#                 lgb.callback.early_stopping(stopping_rounds=100),
#                 lgb.callback.log_evaluation(period=100),
#             ],
#         )
    
        time_cost = time.time() - now_time
        time_cost_list.append(time_cost)

        #logger.info(f"cost time {time_cost}")
#         logger.info(f"cost time {str(datetime.timedelta(seconds=time_cost))}")

        models.append(xgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        xgb_model.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")
#         model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
#         xgb_model.save_model(model_filename)
#         logger.info(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        # fold_predictions = lgb_model.predict(df_fold_valid[feature_columns])
        # fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
        # scores.append(fold_score)
        # logger.info(f":LGB Fold {i+1} MAE: {fold_score}")

        #------------XGB--------------#
        fold_predictions = xgb_model.predict(df_fold_valid[feature_columns])
        fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
        scores.append(fold_score)
        print(f":LGB Fold {i+1} MAE: {fold_score}")
        

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    #average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))
#     average_best_iteration = int(np.mean([model.best_iteration for model in models]))
    # Update the xgb_params with the average best iteration
    final_model_params = xgb_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1
    gc.collect()

    for i in range(num_model):
        break
        now_time = time.time()
        final_model = xgb.XGBRegressor(**final_model_params)
        #final_model.fit(
        #    df_train_feats[feature_columns],
        #    df_train['target'],
        #    callbacks=[
        #        lgb.callback.log_evaluation(period=100),
        #    ],
        #)
        final_model.fit(
            df_train_feats[feature_columns],
            df_train['target'],
            eval_metric='mae',
        )
        # Append the final model to the list of models
#         models.append(final_model)
        # save final model
        model_filename = os.path.join(model_save_path, f'doblez_final.txt')
        final_model.save_model(model_filename)
        print(f"Model for fold final saved to {model_filename}")
        time_cost = time.time() - now_time
        time_cost_list.append(time_cost)
        gc.collect()

XGB_average_mae = np.mean(scores)
time_cost_all = sum(time_cost_list)
print(f"Average MAE across all folds: {XGB_average_mae}")
print(f"Time cost all folds: {time_cost_all/60:.2f} min")
gc.collect()
print('Using total Memory=', get_used_memory_MB(),"MB")

# **Submission**

In [ ]:
# load models
Load_Model = True
if Load_Model:
    num_folds = 10
    premodel_save_path = '/kaggle/input/xgb-200feat-5-fold/modelitos_para_despues' 
    for i in range(1, num_folds + 1):
        model_filename = os.path.join(premodel_save_path, f'doblez_{i}.txt')
        if os.path.exists(model_filename):
            loaded_model = xgb.Booster(model_file=model_filename)
    #         loaded_models.append(loaded_model)
            models.append(loaded_model)
            print(f"Model for fold {i} loaded from {model_filename}")
        else:
            print(f"Model file {model_filename} not found.")

    premodel_save_path = '/kaggle/input/xgb-200final/modelitos_para_despues' 
    model_filename = os.path.join(premodel_save_path, f'doblez_final.txt')
    if os.path.exists(model_filename):
        loaded_model = xgb.Booster(model_file=model_filename)
        models.append(loaded_model)
        print(f"Model for final loaded from {model_filename}")
    else:
        print(f"Model file {model_filename} not found.")

# directly load final model
# models.append(final_model)
print('Using total Memory=', get_used_memory_MB(),"MB")

In [ ]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64
    qps, predictions = [], []
    cache = pd.DataFrame()

    # Weights for each fold model
    if LGB:
        lgb_model_weights = weighted_average(models)
        #cbt_model_weights = weighted_average(models_cbt)
    if XGB:
        xgb_model_weights = weighted_average(models)
    
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        feat = generate_all_features(cache)[-len(test):]
        print(f"Feat Shape is: {feat.shape}")

        # Generate predictions for each model and calculate the weighted average
        if LGB:
            lgb_predictions = np.zeros(len(test))
            for model, weight in zip(models, lgb_model_weights):
                lgb_predictions += weight * model.predict(feat[feature_columns])
        if XGB:
            xgb_predictions = np.zeros(len(test))
            for model, weight,m_index in zip(models, xgb_model_weights,range(1,num_folds+1)):
#                 xgb_predictions += weight * model.predict(feat[feature_columns])
                if m_index<11:
                    # n-fold has attribute "iteration_range" to identify the best iteration
                    xgb_predictions += weight * model.predict(xgb.DMatrix(feat[feature_columns]),iteration_range=(loaded_model.best_iteration, loaded_model.best_iteration+1))
                else:
                    # final model has't best iteration
                    xgb_predictions += weight * model.predict(xgb.DMatrix(feat[feature_columns]))

#         predictions = lgb_predictions
        predictions = xgb_predictions
        
        #Using mean predictions rather than zero sum
        final_predictions = predictions - np.mean(predictions)
        clipped_predictions = np.clip(final_predictions, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))

    time_cost = 1.146 * np.mean(qps)
    print(f"The code will take approximately {np.round(time_cost, 4)} hours to reason about")
